In [121]:
import LowLevelFEM as FEM
using LowLevelFEM
using SparseArrays

gmsh.initialize()

In [122]:
#gmsh.open("wave_in_rectangle.geo")
gmsh.open("wave_in_rectangle_2.geo")

Info    : Reading 'wave_in_rectangle_2.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Line)
Info    : [ 80%] Meshing curve 6 (Line)
Info    : [ 90%] Meshing curve 7 (Line)
Info    : Done meshing 1D (Wall 0.00109634s, CPU 0.001082s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.0282086s, CPU 0.021987s)
Info    : 40401 nodes 80807 elements
Info    : Done reading 'wave_in_rectangle_2.geo'


In [123]:
mat = FEM.material("body", E=2.e5, ν=0.3, ρ=7.85e-9)
problem = FEM.Problem([mat], type="PlaneStress")

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 401)


LowLevelFEM.Problem("wave_in_rectangle_2", "PlaneStress", 2, [("body", 200000.0, 0.3, 7.85e-9)], 1.0, 40401)

In [124]:
supp1 = FEM.displacementConstraint("left", ux=0)
supp2 = FEM.displacementConstraint("middle", uy=0)
load = FEM.load("source", fx=-1);

In [125]:
dof = problem.non * problem.dim
u0 = zeros(dof)
v0 = zeros(dof);

In [126]:
K = FEM.stiffnessMatrix(problem)
M = FEM.massMatrix(problem)
f = FEM.loadVector(problem, [load])
C = spzeros(dof, dof)
dof

80802

In [127]:
FEM.applyBoundaryConditions!(problem, K, M, C, f, [supp1, supp2]);

In [128]:
Tₘᵢₙ = FEM.smallestPeriodTime(K, M)

2.209812832939239e-7

In [129]:
u1, v1, t1 = FEM.CDM(K, M, C, f, u0, v0, 2Tₘᵢₙ, Tₘᵢₙ / π)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 7.034052713403693e-8, 1.4068105426807386e-7, 2.110215814021108e-7, 2.813621085361477e-7, 3.517026356701846e-7, 4.2204316280422154e-7])

In [130]:
sts = length(t1)
u0 = u1[:, sts]
v0 = v1[:, sts]

f .*= 0
u2, v2, t2 = FEM.CDM(K, M, C, f, u0, v0, 30Tₘᵢₙ, Tₘᵢₙ / π)

([0.0 0.0 … 8.008372408890389e-32 5.9006412627521285e-30; 0.0 0.0 … 2.4076661029907953e-31 1.8025938328316598e-29; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 1.1385146991619907e-24 8.274827863561356e-23; 0.0 0.0 … 3.422871850822049e-24 2.5284387880868597e-22; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 7.034052713403693e-8, 1.4068105426807386e-7, 2.110215814021108e-7, 2.813621085361477e-7, 3.517026356701846e-7, 4.2204316280422154e-7, 4.923836899382585e-7, 5.627242170722954e-7, 6.330647442063324e-7  …  5.9789448063931425e-6, 6.04928533352718e-6, 6.119625860661217e-6, 6.189966387795254e-6, 6.260306914929291e-6, 6.330647442063328e-6, 6.4009879691973655e-6, 6.471328496331403e-6, 6.54166902346544e-6, 6.612009550599477e-6])

In [131]:
gmsh.option.setNumber("General.Trackball", 0)
gmsh.option.setNumber("General.RotationX", 298)
gmsh.option.setNumber("General.RotationY", 346)
gmsh.option.setNumber("General.RotationZ", 332)

In [132]:
vvec1 = FEM.showDoFResults(problem, v1, t=t1, "vvec", name="v1(t)", visible=false)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$vvec1].NormalRaise", 5)
gmsh.option.setNumber("View[$vvec1].VectorType", 5)
gmsh.option.setNumber("View[$vvec1].IntervalsType", 3)
gmsh.option.setNumber("View[$vvec1].NbIso", 20)

In [133]:
vvec2 = FEM.showDoFResults(problem, v2, t=t2, "vvec", name="v2(t)", visible=true)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$vvec2].NormalRaise", 5)
gmsh.option.setNumber("View[$vvec2].VectorType", 5)
gmsh.option.setNumber("View[$vvec2].IntervalsType", 3)
gmsh.option.setNumber("View[$vvec2].NbIso", 20)

In [134]:
S2 = FEM.solveStress(problem, u2)

LowLevelFEM.StressField([[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]  …  [0.0 0.0 … -1.7450385911531523e-18 -1.9593176413955226e-17; 0.0 0.0 … -8.251751973711184e-19 -9.309084656255614e-18; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.

In [135]:
s2 = FEM.showStressResults(problem, S2, "seqv", name="σ(t)", t=t2, visible=false, smooth=true)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$s2].NormalRaise", 100)
#gmsh.option.setNumber("View[$s2].VectorType", 5)
gmsh.option.setNumber("View[$s2].IntervalsType", 3)
gmsh.option.setNumber("View[$s2].NbIso", 20)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


In [136]:
#seqv = FEM.showStressResults(problem, S2, "seqv", name="σeqv(t)", t=t2, visible=false, smooth=true)
#gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
#gmsh.option.setNumber("View[$seqv].NormalRaise", 100)
##gmsh.option.setNumber("View[$seqv].VectorType", 5)
#gmsh.option.setNumber("View[$seqv].IntervalsType", 3)
#gmsh.option.setNumber("View[$seqv].NbIso", 20)

In [137]:
FEM.plotOnPath(problem, "path", step=1im, s2, 500)

3

In [138]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.2
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.6.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [139]:
dataType, tags, data, time, numComponents = gmsh.view.getModelData(seqv, 0)

In [ ]:
gmsh.finalize()